In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import sys
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Define the current directory if __file__ is not available
current_dir = os.getcwd()  # Gets the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Moves one level up

# Add the parent directory to the Python path
sys.path.insert(0, parent_dir)

from preprocessing import *

In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# Define the path to the parent directory
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Move one level up

In [ ]:
X = os.path.join(data_dir, "data/X_matrix_YPD_doublingtime.csv")
Y = os.path.join(data_dir, "data/y_YPD_doublingtime.csv")

print("Loading the data...")

# Load the column names (header)
column_names = np.genfromtxt(X, delimiter=',', max_rows=1, dtype=str)[1:]  # Skip the first column if it's row names

# Read the first line of the file to determine the number of columns
with open(X_file, 'r') as f:
    first_line = f.readline()
    end_column_index = len(first_line.split(','))  # Count the number of columns in the first line

# Load the row names (index) from the first column and the data (excluding first column)
data = np.loadtxt(X, delimiter=',', skiprows=1, usecols=range(1, end_column_index))
row_names = np.loadtxt(X, delimiter=',', skiprows=1, usecols=0, dtype=str)

# Create the DataFrame
X_file = pd.DataFrame(data, index=row_names, columns=column_names)
X_file = pd.DataFrame(data, columns=column_names)
y_file = pd.read_csv(Y)

X_data = X_file.drop(columns=["Yeast_ID"])  # Remplacer les valeurs manquantes par 0 dans X
y_data = y_file.drop(columns=["Yeast_ID"])
print(f"Dimensions de X : {X_data.shape}")
print(f"Dimensions de Y : {y_data.shape}")

Loading the data...


ValueError: invalid column index 347953 at row 1 with 347953 columns

In [ ]:
X_data_f, y_data_f = preprocessed_data(X_data, y_data)

In [ ]:
# Division des données en ensembles d'entraînement et de test
print("Division des données en ensemble d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(X_data_f, y_data_f, test_size=0.2, random_state=42)

# Affichage des dimensions des ensembles pour validation
print(f"Dimensions de X_train: {X_train.shape}")
print(f"Dimensions de X_test: {X_test.shape}")
print(f"Dimensions de y_train: {y_train.shape}")
print(f"Dimensions de y_test: {y_test.shape}")

In [ ]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize data
def tokenize_data(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize_data(X_train.tolist())
test_encodings = tokenize_data(X_test.tolist())

In [ ]:
# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
# Prepare datasets
train_dataset = CustomDataset(train_encodings, y_train.tolist())
test_dataset = CustomDataset(test_encodings, y_test.tolist())

In [ ]:
# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(y_data)))
model.to(device)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directory for logging
    evaluation_strategy="epoch",     # Evaluate at each epoch
    save_strategy="epoch",           # Save model at each epoch
    load_best_model_at_end=True      # Load the best model at the end
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
trainer.evaluate()

In [ ]:
# Save the model
model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_model")

In [ ]:
# Predictions
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1)

In [ ]:
# Save predictions to CSV
results_df = pd.DataFrame({
    "true_label": y_test.tolist(),
    "predicted_label": y_pred.tolist()
})
results_df.to_csv("results/y_test_predicted_BERT.csv", index=False)

print("Training complete, and predictions saved.")